In [109]:
import os
from typing import List, Dict, Optional, Union, Any


from langchain.agents import load_tools
from langchain.llms import OpenAI, HuggingFaceHub, Cohere
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts.chat import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.memory import ConversationBufferMemory

from langchain.chains import LLMChain, ConversationChain
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI


from utils import get_local_keys, get_prompts_templates, load_model, load_google_search_tool


In [80]:
keys = get_local_keys()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = keys["huggingface_hub_token"]
os.environ["OPENAI_API_KEY"]=keys["openai_api_key"]

In [81]:
chat = ChatOpenAI(temperature=0.5, max_tokens=512)

In [82]:
anon_text = "PERSON_FIRSTNAME_1 is famous for being an LOCATION_1 songwriter and singer. PRONOUN is from LOCATION_3 in LOCATION_2 and this can be heard heavily in PRONOUN accent even while singing. PERSON_LASTNAME_1!e love PRONOUN music and PRONOUN is famous worldwide. PRONOUN performs live and tours the world with an amazing voice. PRONOUN is loved for PRONOUN unique and powerful voice as well as PRONOUN song writing. PRONOUN songs and albums often about breakups and heartache which many people can relate to. NUMERIC_1 of PRONOUN most famous songs is someone like you. PRONOUN is still only young but has numerous albums that hAve become platinum selling. PRONOUN is now happily married with a child after losing a significant amount of weight which many people were shocked by. PRONOUN is very down to earth and relatable in PRONOUN music"

In [108]:
# Adding memory and conversation chain
conv_base_template = """
    This is a secret conversation between a human and an AI. The AI is helping the user validate their annonimization process, mimicing an intruder

    Current conversation:
    {history}
    Human: {input}
    AI Assistant:
    """
conv_base_prompt = PromptTemplate(input_variables=["history", "input"], template=conv_base_template)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory(return_messages=True),
    prompt=conv_base_prompt
)

In [110]:
template_human_first = """Task: please re-identify the person in the following anonymized text.
    {format_instructions}
    {anon_text}
    """
class FirstTry(BaseModel):
            name: str = Field(description="Name of the person or FAIL")
            score: float = Field(description="The score or re-identifiabilization, 0 is very easy to re-identidy and 1 is impossible")
            charachteristics: List[str] = Field(description="A list of 3 unique and identifying charachteristics names of the person, short answers")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=FirstTry)
human_prompt_first_parser = HumanMessagePromptTemplate.from_template(template=template_human_first, output_parser=parser)
prompt_cur = human_prompt_first_parser.format(anon_text=anon_text, format_instructions=parser.get_format_instructions())
in_text = prompt_cur.content
in_text


'Task: please re-identify the person in the following anonymized text.\n    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "Name of the person or FAIL", "type": "string"}, "score": {"title": "Score", "description": "The score or re-identifiabilization, 0 is very easy to re-identidy and 1 is impossible", "type": "number"}, "charachteristics": {"title": "Charachteristics", "description": "A list of 3 unique and identifying charachteristics names of the person, short answers", "type": "array", "items": {"type": "string"}}}, "req

In [105]:
conversation.memory.clear()
conversation.memory.chat_memory

ChatMessageHistory(messages=[])

In [106]:
response_first = conversation.predict(input=in_text)
response_first

'{"name": "Adele", "score": 0.8}'

In [103]:
parser.parse(response_first)

FirstTry(name='Adele', score=0.8)

# Some Tries

In [91]:
chat_prompt_first = ChatPromptTemplate.from_messages([system_prompt_first, human_prompt_first_parser])
messages = chat_prompt_first.format_prompt(anon_text=anon_text, format_instructions=parser.get_format_instructions()).to_messages()
ans_first_parser = chat(messages).content
output_first = parser.parse(ans_first_parser)
print(output_first)
print(output_first.name)

name='Adele' score=0.8
Adele


In [92]:
# Shortenning
chat_prompt_first = ChatPromptTemplate(
    messages=[system_prompt_first, human_prompt_first_parser],
    input_variables=["anon_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    output_parser=parser,
)
messages = chat_prompt_first.format_prompt(anon_text=anon_text).to_messages()
ans_first_parser = chat(messages).content
output_first = parser.parse(ans_first_parser)
print(output_first)
print(output_first.name)

name='Adele' score=0.8
Adele


In [99]:
# Adding to a chain
chain = LLMChain(llm=chat, prompt=chat_prompt_first)
response = chain.predict_and_parse(anon_text=anon_text)
response

c:\Users\golan\anaconda3\envs\AMI\Lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


FirstTry(name='Adele', score=0.8)

In [ ]:
chat_prompt_first = ChatPromptTemplate(
    messages=[system_prompt_first, human_prompt_first_parser],
    input_variables=["anon_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    output_parser=parser,
)
prompt_for_conv_chain = chat_prompt_first.format_prompt(anon_text=anon_text, format_instructions=parser.get_format_instructions()).to_string()
 #### DONT Knows how to
 # Maybe CONVERSATION chain is not the right one
 # Maybe SequentialChain?

In [ ]:
### Add Memory

In [56]:
template_sys_first = """You are a helpful, respectful and honest assistant.
You are helping the user test their anonimization scheme, by mimicing an Intruder."""
prompt_sys_first = SystemMessagePromptTemplate.from_template(template_sys_first)
template_human_first = HumanMessagePromptTemplate.from_template("{question}")

In [62]:
prompt = ChatPromptTemplate(
    messages=[
        prompt_sys_first,
        MessagesPlaceholder(variable_name="chat_history"),
        template_human_first
    ],
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

In [63]:
conversation.memory.clear()
conversation.memory.chat_memory

ChatMessageHistory(messages=[])

In [64]:
template_ask = """Task: please re-identify the person in the following anonymized text.
    {anon_text}
    """
anon_text = "PERSON_FIRSTNAME_1 is famous for being an LOCATION_1 songwriter and singer. PRONOUN is from LOCATION_3 in LOCATION_2 and this can be heard heavily in PRONOUN accent even while singing. PERSON_LASTNAME_1!e love PRONOUN music and PRONOUN is famous worldwide. PRONOUN performs live and tours the world with an amazing voice. PRONOUN is loved for PRONOUN unique and powerful voice as well as PRONOUN song writing. PRONOUN songs and albums often about breakups and heartache which many people can relate to. NUMERIC_1 of PRONOUN most famous songs is someone like you. PRONOUN is still only young but has numerous albums that hAve become platinum selling. PRONOUN is now happily married with a child after losing a significant amount of weight which many people were shocked by. PRONOUN is very down to earth and relatable in PRONOUN music"
prompt_ask = HumanMessagePromptTemplate.from_template(template_ask)
prompt_cur = prompt_ask.format(anon_text=anon_text)
in_text = prompt_cur.content


In [66]:
conversation.predict(question=in_text)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful, respectful and honest assistant.
You are helping the user test their anonimization scheme, by mimicing an Intruder.
Human: Task: please re-identify the person in the following anonymized text.
    PERSON_FIRSTNAME_1 is famous for being an LOCATION_1 songwriter and singer. PRONOUN is from LOCATION_3 in LOCATION_2 and this can be heard heavily in PRONOUN accent even while singing. PERSON_LASTNAME_1!e love PRONOUN music and PRONOUN is famous worldwide. PRONOUN performs live and tours the world with an amazing voice. PRONOUN is loved for PRONOUN unique and powerful voice as well as PRONOUN song writing. PRONOUN songs and albums often about breakups and heartache which many people can relate to. NUMERIC_1 of PRONOUN most famous songs is someone like you. PRONOUN is still only young but has numerous albums that hAve become platinum selling. PRONOUN is now happily married with a child after losing a signifi


> Finished chain.


'Based on the provided text, the person being referred to is likely Adele.'

In [55]:
conversation("Give me 3 unique charecteristics that helped you re-identify the correct person. ")


{'question': 'Give me 3 unique charecteristics that helped you re-identify the correct person. ',
 'chat_history': [HumanMessage(content='Task: please re-identify the person in the following anonymized text.\n    PERSON_FIRSTNAME_1 is famous for being an LOCATION_1 songwriter and singer. PRONOUN is from LOCATION_3 in LOCATION_2 and this can be heard heavily in PRONOUN accent even while singing. PERSON_LASTNAME_1!e love PRONOUN music and PRONOUN is famous worldwide. PRONOUN performs live and tours the world with an amazing voice. PRONOUN is loved for PRONOUN unique and powerful voice as well as PRONOUN song writing. PRONOUN songs and albums often about breakups and heartache which many people can relate to. NUMERIC_1 of PRONOUN most famous songs is someone like you. PRONOUN is still only young but has numerous albums that hAve become platinum selling. PRONOUN is now happily married with a child after losing a significant amount of weight which many people were shocked by. PRONOUN is ver

In [68]:
conversation = ConversationChain(
    llm=chat, 
    verbose=True, 
    memory=ConversationBufferMemory(return_messages=True)
)

In [70]:
conversation.predict(input=in_text)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Task: please re-identify the person in the following anonymized text.
    PERSON_FIRSTNAME_1 is famous for being an LOCATION_1 songwriter and singer. PRONOUN is from LOCATION_3 in LOCATION_2 and this can be heard heavily in PRONOUN accent even while singing. PERSON_LASTNAME_1!e love PRONOUN music and PRONOUN is famous worldwide. PRONOUN performs live and tours the world with an amazing voice. PRONOUN is loved for PRONOUN unique and powerful voice as well as PRONOUN song writing. PRONOUN songs and albums often about breakups and heartache which many people can relate to. NUMERIC_1 of PRONOUN most famous songs is someone like you. PRONOUN is still only young bu

'Based on the provided information, the person being referred to in the anonymized text is Adele. Adele is famous for being a British songwriter and singer. She is from London, England, and her British accent can be heard in her singing. Adele is loved for her unique and powerful voice as well as her songwriting skills. Her songs often revolve around breakups and heartache, which many people can relate to. One of her most famous songs is "Someone Like You." Adele has released numerous albums that have achieved platinum status. She is now happily married with a child, and she has also undergone a significant weight loss transformation that surprised many people. Despite her fame, Adele is known for being down to earth and relatable in her music.'